In [1]:
import json
import sys
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install ase
import ase
import itertools
import numpy as np
import multiprocessing as mp
import time
import networkx as nx
from scipy.spatial import distance_matrix
from tqdm.auto import tqdm
#!pip install p_tqdm
from p_tqdm import p_map, p_umap, p_imap, p_uimap

from ase import Atoms, Atom
#from ase.visualize import view
from ase.io import read,write
from ase.constraints import dict2constraint
from ase.calculators.singlepoint import SinglePointCalculator
from ase import neighborlist
from ase.db import connect
from ase.visualize.plot import plot_atoms
#from torch_geometric.utils import from_networkx, to_networkx
#import mendeleev

In [2]:
def Analyse_surface(atoms, adsorbate,radii_multiplier=1.1, skin = 0.25): 
    nl = ase.neighborlist.NeighborList(ase.neighborlist.natural_cutoffs(atoms, radii_multiplier), self_interaction=False, 
                          bothways=True, skin=skin)
    nl.update(atoms)
    adsorbate_atoms = [atom.index for atom in atoms if atom.symbol in adsorbate] 
    # Add all atoms to graph 
    distances = atoms.get_all_distances(mic=True)
    binding_atoms = []
    bonds = []
    for index, atom in enumerate(atoms):
        neighbors, offsets = nl.get_neighbors(index)
        for neighbor, offset in zip(neighbors, offsets):
            if np.all(offset==np.zeros(3)):
                if distances[index][neighbor] > 2.5 and (bool(index in adsorbate_atoms) ^ bool(neighbor in adsorbate_atoms)):            
                    continue
                if (bool(index not in adsorbate_atoms) ^ bool(neighbor not in adsorbate_atoms)) and (index not in adsorbate_atoms):    
                    binding_atoms.append(index)
            bonds.append((index, neighbor))                  
    return binding_atoms, adsorbate_atoms, bonds

def bond_symbol(atoms, a1, a2):
    return "{}{}".format(*sorted((atoms[a1].symbol, atoms[a2].symbol)))
def EN(atoms, a1, a2):
    try:
        avg = (mendeleev.element(atoms[a1].symbol).en_pauling + mendeleev.element(atoms[a2].symbol).en_pauling)/2
    except:
        avg = 0.0
    try:
        dif = abs(mendeleev.element(atoms[a1].symbol).en_pauling - mendeleev.element(atoms[a2].symbol).en_pauling)
    except:
        dif = 0
    return np.array([avg, dif])
def node_symbol(atom):
    return "".format(atom.symbol)
def add_atoms_node(graph, atoms, a1):
    graph.add_node(node_symbol(atoms[a1]), index=a1)
def add_atoms_edge(graph, atoms, a1, a2):
    graph.add_edge(a1,a2,electronegativity = EN(atoms, a1, a2))
def add_atoms_edge_2(graph, atoms, a1, a2, b1,b2):
    graph.add_edge(a1,a2,electronegativity = EN(atoms, b1, b2))  
def Atom2Graph(atoms, adsorbate,radii_multiplier=1.1, skin = 0.25):
    nl = ase.neighborlist.NeighborList(ase.neighborlist.natural_cutoffs(atoms, radii_multiplier), self_interaction=False, bothways=True, skin=skin)
    nl.update(atoms)
    adsorbate_atoms = [atom.index for atom in atoms if atom.symbol in adsorbate]
    # Add all atoms to graph 
    distances = atoms.get_all_distances(mic=True)
    full = nx.Graph()
    full.add_nodes_from(range(len(atoms)))
    for i, feat_dict in full.nodes(data=True):
                feat_dict.update({'symbol':  atoms[i].symbol})
                feat_dict.update({'index':  i})
                feat_dict.update({'node_attrs ':  atom_init[atoms[i].symbol]})
                feat_dict.update({'pos ':  atoms.positions[i]})
    binding_atoms = []
    for index, atom in enumerate(atoms):
        neighbors, offsets = nl.get_neighbors(index)
        for neighbor, offset in zip(neighbors, offsets):
            if np.all(offset==np.zeros(3)):
                if distances[index][neighbor] > 2.5 and (bool(index in adsorbate_atoms) ^ bool(neighbor in adsorbate_atoms)):
                    continue
                if (bool(index not in adsorbate_atoms) ^ bool(neighbor not in adsorbate_atoms)) and (index not in adsorbate_atoms):    
                    binding_atoms.append(index)  
                add_atoms_edge(full, atoms, index, neighbor)                      
    return full,  binding_atoms, adsorbate_atoms 
def Extract_Ads_geom(atms, adsorbate, plot=True):
    binding_atoms, adsorbate_atoms, bonds = Analyse_surface(atms, adsorbate=adsorbate)
    #^^^ returns list of index for ads atom, top site atom indexs
    #and index of all bonds
    
    nl = ase.neighborlist.NeighborList(ase.neighborlist.natural_cutoffs(atms, 1.1), 
                                       self_interaction=False, bothways=True, 
                                       skin=0.25)
    nl.update(atms)

    poss = [atms.positions[i] for i in binding_atoms]
    symbols = [atms[i].symbol for i in binding_atoms]

    distances = atms.get_all_distances(mic=True)

    for atm in  binding_atoms: #get neighborlist of top sites (int)
        neighbors, offsets = nl.get_neighbors(atm)
        
        #Add secondary neighbors too--------------------------
        
        #--------------------------
        for i, offset in zip(neighbors, offsets):
            poss.append(atms.positions[i] + np.dot(offset, atms.get_cell()))
            symbols.append(atms[i].symbol)

    for atm in  adsorbate_atoms:
        poss.append(atms.positions[atm])
        symbols.append(atms[atm].symbol)
    coords = np.vstack(poss) 

    surf_n = Atoms(symbols,positions=coords )
    
    ase.geometry.get_duplicate_atoms(surf_n, cutoff=0.1, delete=True)
    if plot:
        fig, ax = plt.subplots(1,2)
        plot_atoms(surf_n,ax[0],radii=0.9, rotation=('0x,0y,0z'))
        plot_atoms(surf_n,ax[1],radii=0.9, rotation=('-90x,45y,0z'))
    return surf_n#,  full #params
#Random idx extracter

#Not_whole = True -> Randomnly extract int(rands) datapoints from adsorbate dataset
#Not_whole = False -> Extract every datapoint

def get_random_idx(ads,rands,Not_whole=True): #ads: adsorbate (str)
    full_idx=[]                #rands: datapoints(int)
    for i in idx:
        at = db.get(i)
        if max_idx<1500:
            if at.Adsorbate == ads: 
                full_idx.append(i)
        if max_idx>1500:
            if at.adsorbate == ads: 
                full_idx.append(i)
    #randomnly select 200 idx
    if Not_whole:
        rand_idx = random.sample(full_idx,k=rands)
    else:
        rand_idx = full_idx
    print(str(len(full_idx))+' -> '+str(len(rand_idx)))
    return(rand_idx)
def create_alignn(indexs,folder,adsorbate):
    adso = []
    file_names=[]
    ads=[]
    Eads_energies=[]
    for j in adsorbate:
        adso.append(j)        
    
    for i in indexs:    
        at = db.get(i)
        if max_idx<1500:
            if at.Adsorbate == adsorbate: 
                Eads = at.Energy
                atms = db.get_atoms(i)

                surf_n = Extract_Ads_geom(atms, adsorbate=adso,plot=False)
                surf_n.set_cell([10,10,30])

                full_file_name = folder+'/POSCAR_'+adsorbate+'_'+str(i)+'.vasp'
                file_name = 'POSCAR_'+adsorbate+'_'+str(i)+'.vasp'
                Eads_energies.append(Eads)
                file_names.append(file_name)
                ads.append(ads)
                #print(full_file_name)
                write(full_file_name, surf_n,format='vasp')

        if max_idx>1500:
            if at.adsorbate == adsorbate: 
                Eads = at.ads_energy
                atms = db.get_atoms(i)

                surf_n = Extract_Ads_geom(atms, adsorbate=adso,plot=False)
                surf_n.set_cell([10,10,30])

                full_file_name = folder+'/POSCAR_'+adsorbate+'_'+str(i)+'.vasp'
                file_name = 'POSCAR_'+adsorbate+'_'+str(i)+'.vasp'
                Eads_energies.append(Eads)
                file_names.append(file_name)
                ads.append(ads)
                write(full_file_name, surf_n,format='vasp')
                
    dic={'files':file_names,'Eads':Eads_energies}
    df = pd.DataFrame(dic)#'''
    #df=0
    return df 

In [ ]:
print('Enter folder name:')
folder = input()
os.mkdir(folder)

print('specify adsorbate molecule (CO/CHO/COOH/O/OH)')
molecule = input()

#get max index of ase db
dbs = ['CO2RR.db','ORR.db']
co2rr=['CO','CHO','COOH']
orr  =['O','OH']
if molecule in co2rr:
    db = connect(dbs[0])
if molecule in orr:
    db = connect(dbs[1])
max_idx = 0
for i in range(10000):
    try:
        at = db.get(i+1)
    except:
        continue
    else:
        max_idx = i+1
idx=[]
for i in range(max_idx):
    at = db.get(i+1)
    idx.append(i+1)
    
print('Datapoints extracted from db: '+str(len(idx)))

x = get_random_idx(molecule,200,Not_whole=False)
df = create_alignn(x,folder,molecule)

df.to_csv(folder+'/id_prop.csv')

Enter folder name:
idk
specify adsorbate molecule (CO/CHO/COOH/O/OH)
CO
Datapoints extracted from db: 710
194 -> 194


In [8]:
!ls

CO2RR.db                Dataset_generator.ipynb toying.ipynb
COOH                    ORR.db
